In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
import scipy.integrate as integrate
import endf
import pandas as pd
import os
import csv
fig,ax=plt.subplots()

In [ ]:
element_symbols=['H','He','Li','Be','B','C','N','O','F','Ne','Na','Mg','Al','Si','P','S','Cl','Ar','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Ga','Ge','As','Se','Br','Kr','Rb','Sr','Y','Zr','Nb','Mo','Tc','Ru','Rh',	'Pd','Ag','Cd','In','Sn','Sb','Te',	'I','Xe','Cs','Ba','La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb','Lu','Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg','Tl','Pb','Bi','Po','At','Rn','Fr','Ra','Ac','Th','Pa','U','Np','Pu','Am','Cm','Bk','Cf','Es','Fm','Md','No']

# Set up units (working in CGS system)
barn=1.e-24 #cm^2

# MT IDs of reactions under consideration
reaction_ids=[16,17,18,102,103]
reaction_strs=["(n,2n)","(n,3n)","(n,fission)","(n,$\gamma$)","(n,p)"]

burnups=["BoL","BoC","EoC","EoL"]




In [36]:
# csv file with the edges of the energy groups, converted MeV --> eV
neutronspec_edges = 1e6*genfromtxt('/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/MPR multizone/neutronspec_edges_scale252.csv', delimiter=',')


n_energy_bins=len(neutronspec_edges)-1


neutronspec_flux_inner = genfromtxt('/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/MPR multizone/neutronspec_flux_inner.csv', delimiter=',')

neutronspec_flux_inner=neutronspec_flux_inner[1:,:]

neutronspec_flux_outer = genfromtxt('/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/MPR multizone/neutronspec_flux_outer.csv', delimiter=',')

neutronspec_flux_outer=neutronspec_flux_outer[1:,:]

neutronspec_flux_middle = genfromtxt('/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/MPR multizone/neutronspec_flux_middle.csv', delimiter=',')

neutronspec_flux_middle=neutronspec_flux_middle[1:,:]



#neutronspec_flux_allburnup=neutronspec_flux_outer


outfile_inner='/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/MPR multizone/onegroup_xs_innerzone.csv'
outfile_middle='/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/MPR multizone/onegroup_xs_middlezone.csv'
outfile_outer='/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/MPR multizone/onegroup_xs_outerzone.csv'


#fields = ['Burnup','Parent ZAI', 'Reaction ID', 'Daughter ZAI', 'Cross-section (b)']

# reactions_array_inner=[['Burnup','Parent ZAI', 'Reaction ID', 'Daughter ZAI', 'Cross-section (b)']]
# reactions_array_middle=[['Burnup','Parent ZAI', 'Reaction ID', 'Daughter ZAI', 'Cross-section (b)']]
# reactions_array_outer=[['Burnup','Parent ZAI', 'Reaction ID', 'Daughter ZAI', 'Cross-section (b)']]




In [ ]:
# Get JEFF 3.3 reaction cross-sections

# Select element, isotope and reaction (by MT ID)
lib_dir='/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/nuclear data/JEFF/'

jeff_reactions = genfromtxt('/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/MPR tests/JEFF_reactions.csv', delimiter=',')

n_reactions=len(jeff_reactions[:,0])
n_good_reactions=n_reactions

firstline=1


for i in range(n_reactions):
    parent_ZAI=jeff_reactions[i,0]

    test_reaction_id=jeff_reactions[i,1]

    meta_flag=np.mod(parent_ZAI,10)
    #print(meta_flag)
    #print("%i, %i" % (parent_ZAI,test_reaction_id))
    parent_ZA=parent_ZAI-meta_flag

    parent_Z=int(parent_ZA/10000)
    parent_A=int(((parent_ZA)-(10000*parent_Z))/10)

    if(meta_flag==1): meta_str="m"
    else: meta_str="g"

    element=element_symbols[parent_Z-1]
    filename="%i-%s-%i%s-900.ace" % (parent_Z,element,parent_A,meta_str)

    infile_str=lib_dir+filename

    table = endf.ace.get_table(infile_str)

    test_reaction = (table.interpret().reactions[test_reaction_id].xs)['900K']



    d1 = {'Energy': test_reaction.x, 'Sigma': test_reaction.y}
    test_df=pd.DataFrame(data=d1)

    # group cross-section data according to the neutron spec group structure
    grouped_xs_df=test_df.groupby(pd.cut(test_df["Energy"],neutronspec_edges),observed=False).mean()

    grouped_xs_df=grouped_xs_df.Sigma.reset_index()
    grouped_xs=grouped_xs_df["Sigma"].copy()


    # if all NaN (i.e. no overlap between xs data and neutronspec data) skip reaction
    if(sum(np.isnan(grouped_xs))==len(grouped_xs)): 
        #print("%i, %i" % (parent_ZAI,test_reaction_id))
        n_good_reactions-=1
        continue

    # if last value is NaN - replace with zeros, count back and replace with zero until a non-NaN value is found
    if(np.isnan(grouped_xs[len(grouped_xs)-1])==1):
        stop_flag=0
        index=len(grouped_xs)-1
        print(index)
        while(stop_flag==0):
            grouped_xs[index]=0
            if(np.isnan(grouped_xs[index-1])==1):
                index-=1
            else:
                stop_flag=1

    # if first value is NaN - replace with zeros, count forward and replace with zero until a non-NaN value is found
    if(np.isnan(grouped_xs[0])==1):
        stop_flag=0
        index=0
        while(stop_flag==0):
            grouped_xs[index]=0
            if(np.isnan(grouped_xs[index+1])==1):
                index+=1
            else:
                stop_flag=1


    # check if any NaN values remain
    any_nan=0
    if(np.sum(np.isnan(grouped_xs))>0): any_nan=1

    while(any_nan==1):
        # count through NaN values, replace each with interpolation from the two closest values
        for i in range(1,len(grouped_xs)-1):
            if np.isnan(grouped_xs[i]):
                lb=1
                while((np.isnan(grouped_xs[i-lb])==1) & ((i-lb)>1)):
                    lb+=1
                ub=1
                while((np.isnan(grouped_xs[i+ub])==1) & ((i+ub)<len(grouped_xs)-1)):
                    ub+=1
                grouped_xs[i]=0.5*(grouped_xs[i-lb]+grouped_xs[i+ub])
        if(np.sum(np.isnan(grouped_xs))==0): any_nan=0   


    for b in range(4):
        neutronspec_flux_inner_singleburnup=neutronspec_flux_inner[:,b]
        neutronspec_flux_middle_singleburnup=neutronspec_flux_middle[:,b]
        neutronspec_flux_outer_singleburnup=neutronspec_flux_outer[:,b]



        prod_sum_inner=0
        flux_sum_inner=0
        for i in range(len(grouped_xs)):
            if(np.isnan(grouped_xs[i])): grouped_xs[i]=0
            prod_sum_inner+=grouped_xs[i]*neutronspec_flux_inner_singleburnup[i]
            flux_sum_inner+=neutronspec_flux_inner_singleburnup[i]


        prod_sum_middle=0
        flux_sum_middle=0
        for i in range(len(grouped_xs)):
            if(np.isnan(grouped_xs[i])): grouped_xs[i]=0
            prod_sum_middle+=grouped_xs[i]*neutronspec_flux_middle_singleburnup[i]
            flux_sum_middle+=neutronspec_flux_middle_singleburnup[i]


        prod_sum_outer=0
        flux_sum_outer=0
        for i in range(len(grouped_xs)):
            if(np.isnan(grouped_xs[i])): grouped_xs[i]=0
            prod_sum_outer+=grouped_xs[i]*neutronspec_flux_outer_singleburnup[i]
            flux_sum_outer+=neutronspec_flux_outer_singleburnup[i]

        if test_reaction_id==16:
            daughter_Z=parent_Z
            daughter_A=parent_A-1
        if test_reaction_id==17:
            daughter_Z=parent_Z
            daughter_A=parent_A-2
        if test_reaction_id==102:
            daughter_Z=parent_Z
            daughter_A=parent_A+1
        if test_reaction_id==103:
            daughter_Z=parent_Z-1
            daughter_A=parent_A
        if test_reaction_id==18:
            daughter_Z=0
            daughter_A=0
        daughter_ZAI=(daughter_Z*10000)+(daughter_A*10)
        #print("%s,%i,%i,%i,%.5e" % (burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum/flux_sum)))

        if(firstline==1):
            reactions_array_inner_JEFF=[[burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_inner/flux_sum_inner)]]
            reactions_array_middle_JEFF=[[burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_middle/flux_sum_middle)]]
            reactions_array_outer_JEFF=[[burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_outer/flux_sum_outer)]]
            firstline=0
        else:
            reactions_array_inner_JEFF.append([burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_inner/flux_sum_inner)])
            reactions_array_middle_JEFF.append([burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_middle/flux_sum_middle)])
            reactions_array_outer_JEFF.append([burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_outer/flux_sum_outer)])


# with open(outfile_inner, 'w') as csvfile:
#     # creating a csv writer object
#     csvwriter = csv.writer(csvfile)
#     # writing the data rows
#     csvwriter.writerows(reactions_array_inner)

# with open(outfile_middle, 'w') as csvfile:
#     # creating a csv writer object
#     csvwriter = csv.writer(csvfile)
#     # writing the data rows
#     csvwriter.writerows(reactions_array_middle)

# with open(outfile_outer, 'w') as csvfile:
#     # creating a csv writer object
#     csvwriter = csv.writer(csvfile)
#     # writing the data rows
#     csvwriter.writerows(reactions_array_outer)
print(n_good_reactions)


In [ ]:
# Get remaining ENDF reaction cross-sections

# Select element, isotope and reaction (by MT ID)
lib_dir='/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/nuclear data/ENDF/Lib80x/Lib80x/'

missing_reactions = genfromtxt('/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/MPR tests/reactions_in_endf_not_jeff.csv', delimiter=',')

n_reactions=len(missing_reactions[:,0])
n_good_reactions=n_reactions
#test_A=115
#test_reaction_id=102

firstline=1
print(n_reactions)
for i in range(n_reactions):
    parent_ZAI=missing_reactions[i,0]

    test_reaction_id=missing_reactions[i,1]

    meta_flag=np.mod(parent_ZAI,10)
    #print("%i, %s, %i" % (parent_ZAI,ZAI_str,meta_flag))
    file_ZA=int(parent_ZAI/10)
    parent_Z=int(parent_ZAI/10000)
    parent_A=int((parent_ZAI-(10000*parent_Z))/10)

    if(meta_flag==1): meta_str="m1_"
    else: meta_str=""

    element=element_symbols[parent_Z-1]
    #print(meta_str)
    #print(file_ZA)

    filename="%s/%s%i.802nc" % (element,meta_str,file_ZA)

    infile_str=lib_dir+filename

    table = endf.ace.get_table(infile_str)

    test_reaction = (table.interpret().reactions[test_reaction_id].xs)['900K']



    d1 = {'Energy': test_reaction.x, 'Sigma': test_reaction.y}
    test_df=pd.DataFrame(data=d1)

    # group cross-section data according to the neutron spec group structure
    grouped_xs_df=test_df.groupby(pd.cut(test_df["Energy"],neutronspec_edges),observed=False).mean()

    grouped_xs_df=grouped_xs_df.Sigma.reset_index()
    grouped_xs=grouped_xs_df["Sigma"].copy()

    # if all NaN (i.e. no overlap between xs data and neutronspec data) skip reaction
    if(sum(np.isnan(grouped_xs))==len(grouped_xs)):
        n_good_reactions-=1
        continue

    # if last value is NaN - replace with zeros, count back and replace with zero until a non-NaN value is found
    if(np.isnan(grouped_xs[len(grouped_xs)-1])==1):
        stop_flag=0
        index=len(grouped_xs)-1
        print(index)
        while(stop_flag==0):
            grouped_xs[index]=0
            if(np.isnan(grouped_xs[index-1])==1):
                index-=1
            else:
                stop_flag=1

    # if first value is NaN - replace with zeros, count forward and replace with zero until a non-NaN value is found
    if(np.isnan(grouped_xs[0])==1):
        stop_flag=0
        index=0
        while(stop_flag==0):
            grouped_xs[index]=0
            if(np.isnan(grouped_xs[index+1])==1):
                index+=1
            else:
                stop_flag=1


    # check if any NaN values remain
    any_nan=0
    if(np.sum(np.isnan(grouped_xs))>0): any_nan=1

    while(any_nan==1):
        # count through NaN values, replace each with interpolation from the two closest values
        for i in range(1,len(grouped_xs)-1):
            if np.isnan(grouped_xs[i]):
                lb=1
                while((np.isnan(grouped_xs[i-lb])==1) & ((i-lb)>1)):
                    lb+=1
                ub=1
                while((np.isnan(grouped_xs[i+ub])==1) & ((i+ub)<len(grouped_xs)-1)):
                    ub+=1
                grouped_xs[i]=0.5*(grouped_xs[i-lb]+grouped_xs[i+ub])
        if(np.sum(np.isnan(grouped_xs))==0): any_nan=0   


    for b in range(4):
        neutronspec_flux_inner_singleburnup=neutronspec_flux_inner[:,b]
        neutronspec_flux_middle_singleburnup=neutronspec_flux_middle[:,b]
        neutronspec_flux_outer_singleburnup=neutronspec_flux_outer[:,b]



        prod_sum_inner=0
        flux_sum_inner=0
        for i in range(len(grouped_xs)):
            if(np.isnan(grouped_xs[i])): grouped_xs[i]=0
            prod_sum_inner+=grouped_xs[i]*neutronspec_flux_inner_singleburnup[i]
            flux_sum_inner+=neutronspec_flux_inner_singleburnup[i]


        prod_sum_middle=0
        flux_sum_middle=0
        for i in range(len(grouped_xs)):
            if(np.isnan(grouped_xs[i])): grouped_xs[i]=0
            prod_sum_middle+=grouped_xs[i]*neutronspec_flux_middle_singleburnup[i]
            flux_sum_middle+=neutronspec_flux_middle_singleburnup[i]


        prod_sum_outer=0
        flux_sum_outer=0
        for i in range(len(grouped_xs)):
            if(np.isnan(grouped_xs[i])): grouped_xs[i]=0
            prod_sum_outer+=grouped_xs[i]*neutronspec_flux_outer_singleburnup[i]
            flux_sum_outer+=neutronspec_flux_outer_singleburnup[i]

        if test_reaction_id==16:
            daughter_Z=parent_Z
            daughter_A=parent_A-1
        if test_reaction_id==17:
            daughter_Z=parent_Z
            daughter_A=parent_A-2
        if test_reaction_id==102:
            daughter_Z=parent_Z
            daughter_A=parent_A+1
        if test_reaction_id==103:
            daughter_Z=parent_Z-1
            daughter_A=parent_A
        if test_reaction_id==18:
            daughter_Z=0
            daughter_A=0
        daughter_ZAI=(daughter_Z*10000)+(daughter_A*10)
        #print("%s,%i,%i,%i,%.5e" % (burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum/flux_sum)))
        if(firstline==1):
            reactions_array_inner_ENDF=[[burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_inner/flux_sum_inner)]]
            reactions_array_middle_ENDF=[[burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_middle/flux_sum_middle)]]
            reactions_array_outer_ENDF=[[burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_outer/flux_sum_outer)]]
            firstline=0
        else:
            reactions_array_inner_ENDF.append([burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_inner/flux_sum_inner)])
            reactions_array_middle_ENDF.append([burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_middle/flux_sum_middle)])
            reactions_array_outer_ENDF.append([burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_outer/flux_sum_outer)])



# with open(outfile_inner, 'w') as csvfile:
#     # creating a csv writer object
#     csvwriter = csv.writer(csvfile)
#     # writing the data rows
#     csvwriter.writerows(reactions_array_inner)

# with open(outfile_middle, 'w') as csvfile:
#     # creating a csv writer object
#     csvwriter = csv.writer(csvfile)
#     # writing the data rows
#     csvwriter.writerows(reactions_array_middle)

# with open(outfile_outer, 'w') as csvfile:
#     # creating a csv writer object
#     csvwriter = csv.writer(csvfile)
#     # writing the data rows
#     csvwriter.writerows(reactions_array_outer)

print(n_good_reactions)


In [ ]:
# Get final TENDL reactions
# Select element, isotope and reaction (by MT ID)
lib_dir='/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/nuclear data/TENDL_ace/'

missing_reactions = genfromtxt('/mnt/c/Users/michael.weekes/OneDrive - Newcleo/Documenti/placement 2/orion/MPR multizone/TENDL_reactions.csv', delimiter=',')

n_reactions=len(missing_reactions[:,0])
n_good_reactions=n_reactions

firstline=1
print(n_reactions)
for i in range(n_reactions):

    parent_ZAI=missing_reactions[i,0]

    test_reaction_id=missing_reactions[i,1]

    

    meta_flag=np.mod(parent_ZAI,10)
    #print("%i, %s, %i" % (parent_ZAI,ZAI_str,meta_flag))
    file_ZA=int(parent_ZAI/10)
    parent_Z=int(parent_ZAI/10000)
    parent_A=int((parent_ZAI-(10000*parent_Z))/10)

    if(meta_flag==0): meta_str=""
    else: meta_str="m"

    if (parent_A>=100): A_str="%i" % parent_A
    elif parent_A>=10: A_str="0%i" % parent_A
    else: A_str="00%i" % parent_A

    element=element_symbols[parent_Z-1]
    #print(meta_str)
    #print(file_ZA)

    filename="%s%s%s" % (element_symbols[parent_Z-1],A_str,meta_str)

    infile_str=lib_dir+filename

    table = endf.ace.get_table(infile_str)

    test_reaction = (table.interpret().reactions[test_reaction_id].xs)['294K']



    d1 = {'Energy': test_reaction.x, 'Sigma': test_reaction.y}
    test_df=pd.DataFrame(data=d1)

    # group cross-section data according to the neutron spec group structure
    grouped_xs_df=test_df.groupby(pd.cut(test_df["Energy"],neutronspec_edges),observed=False).mean()

    grouped_xs_df=grouped_xs_df.Sigma.reset_index()
    grouped_xs=grouped_xs_df["Sigma"].copy()

    # if all NaN (i.e. no overlap between xs data and neutronspec data) skip reaction
    if(sum(np.isnan(grouped_xs))==len(grouped_xs)):
        n_good_reactions-=1
        continue

    # if last value is NaN - replace with zeros, count back and replace with zero until a non-NaN value is found
    if(np.isnan(grouped_xs[len(grouped_xs)-1])==1):
        stop_flag=0
        index=len(grouped_xs)-1
        print(index)
        while(stop_flag==0):
            grouped_xs[index]=0
            if(np.isnan(grouped_xs[index-1])==1):
                index-=1
            else:
                stop_flag=1

    # if first value is NaN - replace with zeros, count forward and replace with zero until a non-NaN value is found
    if(np.isnan(grouped_xs[0])==1):
        stop_flag=0
        index=0
        while(stop_flag==0):
            grouped_xs[index]=0
            if(np.isnan(grouped_xs[index+1])==1):
                index+=1
            else:
                stop_flag=1


    # check if any NaN values remain
    any_nan=0
    if(np.sum(np.isnan(grouped_xs))>0): any_nan=1

    while(any_nan==1):
        # count through NaN values, replace each with interpolation from the two closest values
        for i in range(1,len(grouped_xs)-1):
            if np.isnan(grouped_xs[i]):
                lb=1
                while((np.isnan(grouped_xs[i-lb])==1) & ((i-lb)>1)):
                    lb+=1
                ub=1
                while((np.isnan(grouped_xs[i+ub])==1) & ((i+ub)<len(grouped_xs)-1)):
                    ub+=1
                grouped_xs[i]=0.5*(grouped_xs[i-lb]+grouped_xs[i+ub])
        if(np.sum(np.isnan(grouped_xs))==0): any_nan=0   


    for b in range(4):
            neutronspec_flux_inner_singleburnup=neutronspec_flux_inner[:,b]
            neutronspec_flux_middle_singleburnup=neutronspec_flux_middle[:,b]
            neutronspec_flux_outer_singleburnup=neutronspec_flux_outer[:,b]



            prod_sum_inner=0
            flux_sum_inner=0
            for i in range(len(grouped_xs)):
                if(np.isnan(grouped_xs[i])): grouped_xs[i]=0
                prod_sum_inner+=grouped_xs[i]*neutronspec_flux_inner_singleburnup[i]
                flux_sum_inner+=neutronspec_flux_inner_singleburnup[i]


            prod_sum_middle=0
            flux_sum_middle=0
            for i in range(len(grouped_xs)):
                if(np.isnan(grouped_xs[i])): grouped_xs[i]=0
                prod_sum_middle+=grouped_xs[i]*neutronspec_flux_middle_singleburnup[i]
                flux_sum_middle+=neutronspec_flux_middle_singleburnup[i]


            prod_sum_outer=0
            flux_sum_outer=0
            for i in range(len(grouped_xs)):
                if(np.isnan(grouped_xs[i])): grouped_xs[i]=0
                prod_sum_outer+=grouped_xs[i]*neutronspec_flux_outer_singleburnup[i]
                flux_sum_outer+=neutronspec_flux_outer_singleburnup[i]

            if test_reaction_id==16:
                daughter_Z=parent_Z
                daughter_A=parent_A-1
            if test_reaction_id==17:
                daughter_Z=parent_Z
                daughter_A=parent_A-2
            if test_reaction_id==102:
                daughter_Z=parent_Z
                daughter_A=parent_A+1
            if test_reaction_id==103:
                daughter_Z=parent_Z-1
                daughter_A=parent_A
            if test_reaction_id==18:
                daughter_Z=0
                daughter_A=0
            daughter_ZAI=(daughter_Z*10000)+(daughter_A*10)
            #print("%s,%i,%i,%i,%.5e" % (burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum/flux_sum)))
            if(firstline==1):
                reactions_array_inner_TENDL=[[burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_inner/flux_sum_inner)]]
                reactions_array_middle_TENDL=[[burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_middle/flux_sum_middle)]]
                reactions_array_outer_TENDL=[[burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_outer/flux_sum_outer)]]
                firstline=0
            else:
                reactions_array_inner_TENDL.append([burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_inner/flux_sum_inner)])
                reactions_array_middle_TENDL.append([burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_middle/flux_sum_middle)])
                reactions_array_outer_TENDL.append([burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_outer/flux_sum_outer)])


            print("%s,%i,%i,%i,%.3e" % (burnups[b],parent_ZAI,test_reaction_id,daughter_ZAI,(prod_sum_inner/flux_sum_inner)))

#print(reactions_array_inner)

# with open(outfile_inner, 'w') as csvfile:
#     # creating a csv writer object
#     csvwriter = csv.writer(csvfile)
#     # writing the data rows
#     csvwriter.writerows(reactions_array_inner)

# with open(outfile_middle, 'w') as csvfile:
#     # creating a csv writer object
#     csvwriter = csv.writer(csvfile)
#     # writing the data rows
#     csvwriter.writerows(reactions_array_middle)

# with open(outfile_outer, 'w') as csvfile:
#     # creating a csv writer object
#     csvwriter = csv.writer(csvfile)
#     # writing the data rows
#     csvwriter.writerows(reactions_array_outer)

print(n_good_reactions)

In [ ]:
print(reactions_array_inner_TENDL)

In [45]:
with open(outfile_inner, 'x') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
    # writing the data rows
    csvwriter.writerow(['Burnup','Parent ZAI', 'Reaction ID', 'Daughter ZAI', 'Cross-section (b)'])
    csvwriter.writerows(reactions_array_inner_JEFF)
    csvwriter.writerows(reactions_array_inner_ENDF)
    csvwriter.writerows(reactions_array_inner_TENDL)


with open(outfile_middle, 'x') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
    # writing the data rows
    csvwriter.writerow(['Burnup','Parent ZAI', 'Reaction ID', 'Daughter ZAI', 'Cross-section (b)'])
    csvwriter.writerows(reactions_array_middle_JEFF)
    csvwriter.writerows(reactions_array_middle_ENDF)
    csvwriter.writerows(reactions_array_middle_TENDL)

with open(outfile_outer, 'x') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
    # writing the data rows
    csvwriter.writerow(['Burnup','Parent ZAI', 'Reaction ID', 'Daughter ZAI', 'Cross-section (b)'])
    csvwriter.writerows(reactions_array_outer_JEFF)
    csvwriter.writerows(reactions_array_outer_ENDF)
    csvwriter.writerows(reactions_array_outer_TENDL)
